In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 

In [27]:
def plot_costs(costs, learning_rate=0.001):
    plt.plot(np.squeeze(costs))
    plt.ylabel('cost')
    plt.xlabel('iterations (per 50 iterations)')
    plt.title("Learning rate =" + str(learning_rate))
    plt.show()

In [5]:
def relu(Z):
    
    A = np.maximum(0,Z)
    assert(A.shape == Z.shape)
    cache = Z 
    
    return A, cache

In [6]:
def sigmoid(Z):
    
    A = 1/(1+np.exp(-Z))
    cache = Z
    
    return A, cache

In [11]:
def relu_backward(dA, cache):
    
    Z = cache
    dZ = np.array(dA, copy=True) # just converting dz to a correct object.
    dZ[Z <= 0] = 0
    assert (dZ.shape == Z.shape)
    
    return dZ

In [12]:
def sigmoid_backward(dA, cache):
    
    Z = cache
    s = 1/(1+np.exp(-Z))
    dZ = dA * s * (1-s)
    assert (dZ.shape == Z.shape)
    
    return dZ


In [2]:
def initialize_parameters_deep(layer_dims):
    
    np.random.seed(1)
    parameters = {}
    L = len(layer_dims) # number of layers in the network

    for l in range(1, L):
        parameters['W' + str(l)] = np.random.randn(layer_dims[l], layer_dims[l-1]) * 0.01
        parameters['b' + str(l)] = np.zeros((layer_dims[l], 1)) 
        
        assert(parameters['W' + str(l)].shape == (layer_dims[l], layer_dims[l - 1]))
        assert(parameters['b' + str(l)].shape == (layer_dims[l], 1))

        
    return parameters

In [13]:
def linear_forward(A, W, b):
    Z = np.dot(W,A) + b 
    cache = (A,W,b)
    
    return Z, cache 

In [14]:
def linear_activation_forward(A_prev, W, b, activation):
    Z, linear_cach = linear_forward(A_prev,W,b)
    
    if activation == "sigmoid":
        A, activation_cache = sigmoid(Z)
        cache = (linear_cach, activation_cache)
    elif activation == "relu":
        A, activation_cache = relu(Z)
        cache = (linear_cach, activation_cache)
    
    
    return A, cache

In [15]:
def L_model_forward(X, parameters):
    caches = []
    A = X 
    L = len(parameters) // 2 
    
    for l in range(1,L):
        A_prev = A 
        A, cache = linear_activation_forward(A_prev, parameters['W' + str(l)], parameters['b' + str(l)], "relu")
        caches.append(cache)
        
    AL, cache = linear_activation_forward(A, parameters['W' + str(L)], parameters['b' + str(L)], "sigmoid")
    caches.append(cache)
    
    return AL, caches

In [16]:
def compute_cost(AL, Y):
    
    m = Y.shape[1]
    cost = -1/m *(np.dot(Y, np.log(AL).T) + np.dot((1-Y), np.log(1-AL).T))
    cost = np.squeeze(cost)      #  this turns [[1]] into 1).
    
    return cost

In [17]:
def linear_backward(dZ, cache):
    A_prev, W, b = cache
    m = A_prev.shape[1]
    
    dW = 1/m * np.dot(dZ, A_prev.T)
    db = 1/m * np.sum(dZ, axis=1, keepdims=True)
    dA_prev = np.dot(W.T, dZ)
    
    return dW, db, dA_prev 

In [18]:
def linear_activation_backward(dA, cache, activation):
    linear_cache, activation_cache = cache
    
    if activation == "relu":
        dZ = relu_backward(dA, activation_cache)
        dA_prev, dW, db = linear_backward(dZ, linear_cache)
    elif activation == "sigmoid":
        dZ = sigmoid_backward(dA, activation_cache)
        dA_prev, dW, db = linear_backward(dZ, linear_cache)
    
    return dA_prev, dW, db 

In [20]:
def L_model_backward(AL, Y, caches):

    grads = {}
    L = len(caches) 
    m = AL.shape[1]
    Y = Y.reshape(AL.shape) # AL is a y_predicted from forward
    
    dAL = -(np.divide(Y, AL) - np.divide(1 - Y, 1 - AL))
    
   
    current_cache = caches[L-1]
    dA_prev_temp, dW_temp, db_temp = linear_activation_backward(dAL, current_cache, activation='sigmoid')
    grads["dA" + str(L-1)] = dA_prev_temp
    grads["dW" + str(L)] = dW_temp
    grads["db" + str(L)] = db_temp
   
    for l in reversed(range(L-1)):
       
        current_cache = caches[l]
        dA_prev_temp, dW_temp, db_temp = linear_activation_backward(grads['dA' + str(l+1)], current_cache, activation='relu')
        grads["dA" + str(l)] = dA_prev_temp
        grads["dW" + str(l+1)] = dW_temp
        grads["db" + str(l+1)] = db_temp
        

    return grads

In [28]:
def update_parameters(params, grads, learning_rate):
    
    parameters = params.copy()
    L = len(parameters) // 2 

    for l in range(L):
        
        parameters["W" + str(l+1)] = parameters["W" + str(l+1)] - learning_rate * grads["dW" + str(l+1)]
        parameters["b" + str(l+1)] = parameters["b" + str(l+1)] - learning_rate * grads["db" + str(l+1)]
        
    return parameters

In [24]:
layers_dims = [X_train[0], 20, 7, 5, 1]  # 1-input_layer --- 3-hidden_layer --- 1-output_layer     

NameError: name 'X_train' is not defined

In [22]:
def NN_model(X, Y, layers_dims, learning_rate = 0.001, num_iterations = 300, print_cost=False):
    """
    
    1- Initialize parameters / Define hyperparameters
    
    2- Loop for num_iterations: 
    
        a. Forward propagation 
        b. Compute cost function 
        c. Backward propagation 
        d. Update parameters (using parameters, and grads from backprop)
        
    3- Use trained parameters to predict labels
    
    """

    np.random.seed(1)
    costs = []                        
    
    parameters = initialize_parameters_deep(layers_dims)
    
    for i in range(0, num_iterations):
        
        AL, caches = L_model_forward(X, parameters)
        
        cost = compute_cost(AL, Y)
        
        grads = L_model_backward(AL, Y, caches)
        
        parameters = update_parameters(parameters, grads, learning_rate)
        
        if print_cost and i % 100 == 0 or i == num_iterations - 1:  # print cost for each 100-iterations 
            print("Cost after iteration {}: {}".format(i, np.squeeze(cost)))
        if i % 100 == 0 or i == num_iterations:
            costs.append(cost)
    
    return parameters, costs

In [26]:
def predict(X, y, parameters):
    
    m = X.shape[1]
    L = len(parameters) // 2 # number of layers in the neural network
    p = np.zeros((1,m))
    
    probas, caches = L_model_forward(X, parameters)

    for i in range(0, probas.shape[1]):
        if probas[0,i] > 0.5:
            p[0,i] = 1
        else:
            p[0,i] = 0
        
    return p